# Bringing it all together

In this notebook we aim to apply several different stratifications, mixing and vaccination concepts learnt previously into a single model. The goal here is to take everything that we have learnt previously using "toy" code with simplified stratifications/functionality loosely representative of Covid transmission - and extend this to a model that we can be satisfied reasonably replicates most of the most important dynamics of Covid transmission in Malaysia.

A little like notebook number 04, this is an opportunity to take stock and apply the epidemiological principles we have worked through in the preceding notebooks.

## Standard preliminaries
Before we get into building the model, let's start off with some of our standard (or "boilerplate") code to get everything set up.

In [ ]:
# pip install the required packages if running in Colab
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
# Standard imports, plotting option and constant definition
from datetime import datetime, timedelta
from typing import List, Union
import pandas as pd
import plotly.express as px
import numpy as np

from summer.utils import ref_times_to_dti

from summer import CompartmentalModel, Stratification, StrainStratification, Overwrite

# Have an import from AuTuMN here - will need to get rid of this ***
from autumn.core import inputs

pd.options.plotting.backend = "plotly"

COVID_BASE_DATE = datetime(2019, 12, 31)
region = "Malaysia"

In [ ]:
# Get the mixing matrix for Malaysia
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/MYS_matrices.pkl

mixing_matrix = pd.read_pickle("MYS_matrices.pkl", compression='infer')
age_mixing_matrix = mixing_matrix["all_locations"]
# px.imshow(age_mixing_matrix)

In [ ]:
# Get a function to access the Malaysia data if running in Colab
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/get_mys_data.py

import get_mys_data

In [ ]:
from autumn.core.inputs.covid_mys.queries import get_mys_vac_coverage

In [ ]:
age_groups = range(0, 80, 5)

age_pops = pd.Series(
    inputs.get_population_by_agegroup(age_groups, "MYS", None, 2020),
    index=age_groups
)
age_pops.index = age_pops.index.astype(str)

In [ ]:
# ... and use it to get the actual data
df = get_mys_data.fetch_mys_data()
initial_population = get_mys_data.get_initial_population(region)
observations = get_mys_data.get_target_observations(df, region, "cases")

In [ ]:
# Define the model running period and convert to a numeric representation
start_date = datetime(2021, 3, 15)
end_date = start_date + timedelta(days=500)
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [ ]:
# Define a target set of observations to compare against our modelled outputs later
notifications_target = observations["cases_new"]
deaths_target = get_mys_data.get_target_observations(df, region, "deaths")["deaths_new"]

# Model

## Define a model

In [ ]:
unstratified_compartments = ["S", "E", "I", "R", "S2"]

In [ ]:
def build_unstratified_model(parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        parameters: Flow parameters
    Returns:
        A compartmental model currently without stratification applied
    """

    model = CompartmentalModel(
        times=(parameters["start_time"], parameters["end_time"]),
        compartments=unstratified_compartments,
        infectious_compartments=["I"],
        ref_date=COVID_BASE_DATE
    )

    infectious_seed = parameters["infectious_seed"]

    model.set_initial_population(
        distribution=
        {
            "S": initial_population - infectious_seed, 
            "I": infectious_seed
        }
    )
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E",
    )
    
    # Recovered people can also get infected
    model.add_infection_frequency_flow(
        name="reinfection", 
        contact_rate=parameters["contact_rate"], 
        source="S2", 
        dest="E",
    )
    
    # Expose people transition to infected
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )
    
    # Infectious people recover
    model.add_transition_flow(
        name="waning_natural_immunity",
        fractional_rate=parameters["waning_immunity_rate"],
        source="R",
        dest="S2",
    )

    # Add an infection-specific death flow to the I compartment
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")
    
    model.request_output_for_flow(
        "progressions",
        "progression",
    )
    
    def prop_detected(progressions):
        return progressions * parameters["cdr"]
    
    model.request_function_output(
        "notifications",
        func=prop_detected,
        sources=["progressions"],
    )

    return model

In [ ]:
def get_age_stratification(
    compartments_to_stratify: List[str],
    strata: List[str],
    matrix: Union[np.ndarray, callable],
) -> Stratification:
    """
    Create a summer stratification object that stratifies all of the compartments into
    strata, which are intended to represent age bands according to the user inputs.
    This is essentially adapting the model's age stratification approach to the format
    of the mixing matrix, which is a reasonable approach.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify, which should be all the compartments
        strata: The strata to be implemented in the age stratification
        matrix: The mixing matrix we are applying for the age structure
    Returns:
        A summer stratification object to represent age stratification (not yet applied)
    """
    
    if isinstance(matrix, np.ndarray):
        msg = "Mixing matrix is not 2-dimensional"
        assert matrix.ndim == 2, msg

        msg = f"Dimensions of the mixing matrix incorrect: {matrix.shape[0]}, {matrix.shape[1]}, {len(strata)}"
        assert matrix.shape[0] == matrix.shape[1] == len(strata), msg
    
    # Create the stratification, just naming the age groups by their starting value
    strat = Stratification(name="age", strata=strata, compartments=compartments_to_stratify)
    
    age_split_props = age_pops / age_pops.sum()
    strat.set_population_split(age_split_props.to_dict())
    
    # Add the mixing matrix to the stratification
    strat.set_mixing_matrix(matrix)
    
    return strat

In [ ]:
def get_strain_stratification(
    compartments_to_stratify: List[str], 
    voc_params: dict
) -> Stratification:
    """
    Create a summer stratification object that stratifies compartments into
    strata, which are intended to represent infectious disease strains.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify
        voc_params: A dictionary which speicifies the infectiousness and severity of strains
    Returns:
        A summer stratification object to represent strain stratification (not yet applied)
    """
    strata = [
        "delta", 
        "omicron"
    ]
    strat = StrainStratification(name="strain", strata=strata, compartments=compartments_to_stratify)

    # At the start of the simulation, a certain proportion of infected people have the variant strain.
    strat.set_population_split(
        {
            "delta": 1.,
            "omicron": 0.,
        }
    )

    for infection_flow in ["infection", "reinfection"]:
        strat.set_flow_adjustments(
            infection_flow,
            {
                "delta": None,
                "omicron": voc_params["omicron_rel_transmissibility"],
            },
        )

    return strat

In [ ]:
full_dose_coverage = get_mys_vac_coverage(dose="full")
full_dose_coverage.index = ref_times_to_dti(COVID_BASE_DATE, full_dose_coverage.index)
full_dose_coverage.plot.area(title="two-dose vaccination coverage")

In [ ]:
# To save on calculations a little, let's thin out the data
thinning_interval = 7
thinned_full_coverage = full_dose_coverage[::thinning_interval]

def get_prop_of_remaining_covered(old_prop, new_prop):
    return (new_prop - old_prop) / (1. - old_prop)

interval_prop_unvacc_vaccinated = [
    get_prop_of_remaining_covered(
        thinned_full_coverage[i],
        thinned_full_coverage[i + 1],
    ) 
    for i in range(len(thinned_full_coverage) - 1)
]

coverage_times = thinned_full_coverage.index

pd.Series(interval_prop_unvacc_vaccinated, index=coverage_times[1:]).plot(
    title="proportion of remaining unvaccinated vaccinated during each interval"
)

In [ ]:
def get_rate_from_coverage_and_duration(coverage_increment: float, duration: float) -> float:
    assert duration >= 0.0, f"Duration request is negative: {duration}"
    assert 0.0 <= coverage_increment <= 1.0, f"Coverage increment not in [0, 1]: {coverage_increase}"
    return -np.log(1.0 - coverage_increment) / duration

vacc_times_int = (coverage_times - COVID_BASE_DATE).days

interval_lengths = [
    vacc_times_int[i + 1] - vacc_times_int[i] 
    for i in range(len(vacc_times_int) - 1)
]

vaccination_rates = [
    get_rate_from_coverage_and_duration(i, j) for 
    i, j in zip(interval_prop_unvacc_vaccinated, interval_lengths)
]
pd.Series(vaccination_rates, index=coverage_times[1:]).plot(kind="scatter")

In [ ]:
def get_vacc_rate_func(end_times, vaccination_rates):
    def get_vaccination_rate(time, derived_outputs):

        # Identify the index of the first list element greater than the time of interest
        # If there is such an index, return the corresponding vaccination rate
        for end_i, end_t in enumerate(end_times):
            if end_t > time:
                return vaccination_rates[end_i]

        # Return zero if the time is after the last end time
        return 0.0
    return get_vaccination_rate

vacc_rate_func = get_vacc_rate_func(vacc_times_int, vaccination_rates)

In [ ]:
def get_vaccine_stratification(
    compartments_to_stratify: List[str], 
    vaccine_params: dict
) -> Stratification:
    """
    Create a summer stratification object that stratifies compartments into
    strata, which are intended to represent vaccine stratifications.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify
        vaccine_params: A dictionary which speicifies the vaccination-related parameters to implement
    Returns:
        A summer stratification object to represent strain stratification (not yet applied)
    """
    strata = ["vaccinated", "unvaccinated"]
    
    # Create the stratification
    vaccine_strat = Stratification(name="vaccination", strata=strata, compartments=compartments_to_stratify)

    # Create our population split dictionary, whose keys match the strata with 80% vaccinated and 20% unvaccinated
    pop_split = {
        "vaccinated": 0., 
        "unvaccinated": 1.,
    }

    # Set a population distribution
    vaccine_strat.set_population_split(pop_split)

    # Adjusting the death risk associated with vaccination
    vaccine_strat.set_flow_adjustments(
        "infection_death",
        {
            "unvaccinated": None,
            "vaccinated": 1. - vaccine_params["ve_death"],
        }
    )
    
    # Susceptibility
    for infection_flow in ["infection", "reinfection"]:
        vaccine_strat.set_flow_adjustments(
            infection_flow,
            {
                "unvaccinated": None,
                "vaccinated": 1. - vaccine_params["ve_infection"],
            }
        )

    return vaccine_strat

In [ ]:
parameters = {
    "contact_rate": 0.07,
    "progression_rate": 0.3,
    "recovery_rate": 0.2,
    "death_rate": 0.001,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 200.,
    "cdr": 0.12,
    "waning_immunity_rate": 1. / 180.,
}

# Get an unstratified model object
model = build_unstratified_model(parameters)

# Get and apply the age stratification
age_strat = get_age_stratification(
    model.compartments, 
    range(0, 80, 5), 
    mixing_matrix["all_locations"],
)
model.stratify_with(age_strat)

# Get and apply vaccination stratification
vacc_params = {
    "ve_death": 0.9,
    "ve_infection": 0.3,
}

vacc_strat = get_vaccine_stratification(
    model.compartments,
    vacc_params,
)
model.stratify_with(vacc_strat)

# Get and apply the strain stratification
strain_strat = get_strain_stratification(
    compartments_to_stratify=["E", "I"],
    vacc_params,
)
model.stratify_with(strain_strat)


def omicron_seed_func(time, computed_values):
    """
    A simple step function to allow seeding of the new strain
    (Omicron) after the start of the analysis period.
    """    
    if 570 < time < 600:
        return 1.
    return 0.


model.add_importation_flow(
    "omicron_seeding",
    omicron_seed_func,
    "E",
    split_imports=True,
    dest_strata={"strain": "omicron"},
)

for comp in unstratified_compartments:
    model.add_transition_flow(
        name="vaccination",
        fractional_rate=vacc_rate_func,
        source=comp,
        dest=comp,
        source_strata={"vaccination": "unvaccinated"},
        dest_strata={"vaccination": "vaccinated"},
    )

model.request_output_for_compartments(
    "vaccinated",
    unstratified_compartments,
    strata={"vaccination": "vaccinated"},
)
model.request_output_for_compartments(
    "unvaccinated",
    unstratified_compartments,
    strata={"vaccination": "unvaccinated"},
)

model.run()

In [ ]:
comparison_df = pd.DataFrame({
    "modelled": model.get_derived_outputs_df()["notifications"],
    "reported": notifications_target,
})
comparison_df.plot()

In [ ]:
derived_df = model.get_derived_outputs_df()
derived_df[["vaccinated", "unvaccinated"]].plot.area()